In [1]:
!pip install ortools

In [29]:
import pandas as pd
from ortools.sat.python import cp_model
from datetime import datetime, timedelta


In [31]:
courses = pd.read_csv('/Users/nanafrimpong/Library/Mobile Documents/com~apple~CloudDocs/Capstone/Implementation/Sample_Testing_Data_1.csv')
rooms   = pd.read_csv('/Users/nanafrimpong/Library/Mobile Documents/com~apple~CloudDocs/Capstone/Implementation/Room_Information_DataSet.csv')

In [82]:
courses

,Session ID,Course Code,Session Type,Number of Enrollments,Cohort,Lecturer,Duration
0,1,BUSA161,Discussion,45,Section A,Katelyn Aba Dadzie,1:00:00
1,2,BUSA442,Discussion,16,Section A,Brian Botchway,1:00:00
2,3,BUSA423,Discussion,13,Section A,Evans Ghansah,1:00:00
3,4,BUSA224,Discussion,55,Section A,Michael Effah Asamoah,1:00:00
4,5,ECON100,Discussion,29,Section A,Akosua Obeng,1:00:00
...,...,...,...,...,...,...,...
438,439,AS111,Seminar,20,Section O,Adjunct Lecturer,1:30:00
439,440,AS111,Seminar,20,Section P,Adjunct Lecturer,1:30:00
440,441,AS111,Seminar,20,Section Q,Jude Samuel Acquaah,1:30:00
441,442,AS111,Seminar,15,Section I,Yvonne Dewortor,1:30:00


In [80]:
selected_columns = ["Course Code", "Duration", "Cohort"]
filtered_courses = courses[selected_columns]
filtered_courses['Course Code'].unique()

array(['BUSA161', 'BUSA442', 'BUSA423', 'BUSA224', 'ECON100', 'BUSA456',
       'BUSA444', 'BUSA431', 'BUSA 432', 'BUSA350', 'BUSA210', 'BUSA422',
       'BUSA402', 'ECON101', 'BUSA457', 'BUSA304', 'CS111', 'CS221',
       'CS442', 'CS433', 'CS341-LAB', 'CS462', 'CS213', 'IS451', 'IS351',
       'EE421', 'EE454', 'EE341', 'EE451', 'EE320', 'ME212', 'ME422',
       'ME442', 'ME311', 'ME434', 'ME444', 'ENGR311', 'FRENC111',
       'SOAN 322', 'POLS 322', 'SOAN225', 'SOAN251', 'SOAN301', 'SOAN331',
       'MATH141', 'MATH121', 'MATH161', 'MATH122', 'SOAN320',
       'ENGR212-LAB', 'MATH101', 'BUSA321', 'BUSA401_A', 'ECON231',
       'BUSA491', 'CS456', 'CS331', 'CS458', 'CS341', 'CS313',
       'CS410-SEM', 'ME401', 'ENGR112', 'SC113', 'SOAN411', 'CE451',
       'AS111', 'MATH145', 'MATH221', 'SOAN211', 'BUSA411', 'CS330',
       'CS 452', 'ENGR412', 'ENGR461', 'ENGL112', 'MATH152', 'ENGR212',
       'ENGR301', 'SOAN328', 'ENGR401-SEM'], dtype=object)

In [78]:
rooms

,Location,Max Room Capacity
0,Apt Hall 216,74.0
1,Apt Hall 217,75.0
2,Bio Lab,33.0
3,D-Lab 102,58.0
4,Databank Foundation Hall 218,75.0
5,EE lab,52.0
6,Fab Lab 103,31.0
7,Fab Lab 203,72.0
8,Fab Lab 303,75.0
9,Jackson Hall 115,76.0


In [56]:

# Scheduler settings
scheduler_start_time = datetime.strptime("08:00 AM", "%I:%M %p")
scheduler_end_time = datetime.strptime("05:00 PM", "%I:%M %p")
block_durations = [60, 90, 105, 120, 180, 195]  # Minutes
break_duration = 15  # Minutes
days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

# Generate time slots for each day of the week
time_slots_per_day = {}
for day in days_of_week:
    time_slots = []
    current_time = scheduler_start_time
    block_index = 0  # Start with the first block duration

    while current_time < scheduler_end_time:
        block_duration = block_durations[block_index]
        next_time = current_time + timedelta(minutes=block_duration)
        if next_time > scheduler_end_time:
            break
        time_slots.append((current_time, next_time))
        current_time = next_time + timedelta(minutes=break_duration)
        block_index = (block_index + 1) % len(block_durations)

    time_slots_per_day[day] = time_slots

filtered_courses = filtered_courses
rooms = rooms

# Ensure Duration column contains minutes
if filtered_courses["Duration"].dtype == 'object':
    filtered_courses = filtered_courses.copy()  # Create a copy to avoid warnings
    filtered_courses["Duration"] = filtered_courses["Duration"].apply(
        lambda x: int(x.split(":")[0]) * 60 + int(x.split(":")[1])
    )
# Initialize the schedule
schedule = []

# Assign courses to rooms and time slots across all days
for day, time_slots in time_slots_per_day.items():
    for room in rooms.itertuples():
        room_slots = time_slots.copy()  # Separate time slots for each room
        for index, course in filtered_courses.iterrows():
            for slot in room_slots:
                # Check duration fits the time slot
                slot_duration = (slot[1] - slot[0]).seconds // 60
                if course["Duration"] <= slot_duration:
                    # Assign course to room and time slot
                    schedule.append({
                        "Day": day,
                        "Course Code": course["Course Code"],
                        "Cohort": course["Cohort"],
                        "Room": room.Location,
                        "Start Time": slot[0].strftime("%I:%M %p"),
                        "End Time": slot[1].strftime("%I:%M %p")
                    })
                    # Remove the time slot (it's now occupied)
                    room_slots.remove(slot)
                    break

# Ensure all rooms are scheduled for the full timeline across all days
all_slots_schedule = []
for day, time_slots in time_slots_per_day.items():
    for room in rooms.itertuples():
        room_schedule = [entry for entry in schedule if entry['Room'] == room.Location and entry['Day'] == day]
        remaining_slots = [slot for slot in time_slots if all(
            not (slot[0] <= datetime.strptime(entry['Start Time'], "%I:%M %p") < slot[1] or
                 slot[0] < datetime.strptime(entry['End Time'], "%I:%M %p") <= slot[1]) for entry in room_schedule
        )]
        for slot in remaining_slots:
            all_slots_schedule.append({
                "Day": day,
                "Course Code": "Free Slot",
                "Cohort": "N/A",
                "Room": room.Location,
                "Start Time": slot[0].strftime("%I:%M %p"),
                "End Time": slot[1].strftime("%I:%M %p")
            })

# Combine scheduled and free slots
complete_schedule = pd.DataFrame(schedule + all_slots_schedule)
complete_schedule.sort_values(by=["Day", "Room", "Start Time"], inplace=True)

print(complete_schedule)


           Day Course Code     Cohort          Room Start Time  End Time
323     Friday     BUSA224  Section A  Apt Hall 216   01:00 PM  03:00 PM
320     Friday     BUSA161  Section A  Apt Hall 216   08:00 AM  09:00 AM
321     Friday     BUSA442  Section A  Apt Hall 216   09:15 AM  10:45 AM
322     Friday     BUSA423  Section A  Apt Hall 216   11:00 AM  12:45 PM
327     Friday     BUSA224  Section A  Apt Hall 217   01:00 PM  03:00 PM
..         ...         ...        ...           ...        ...       ...
234  Wednesday     BUSA423  Section A  Radichel MPR   11:00 AM  12:45 PM
239  Wednesday     BUSA224  Section A   Science Lab   01:00 PM  03:00 PM
236  Wednesday     BUSA161  Section A   Science Lab   08:00 AM  09:00 AM
237  Wednesday     BUSA442  Section A   Science Lab   09:15 AM  10:45 AM
238  Wednesday     BUSA423  Section A   Science Lab   11:00 AM  12:45 PM

[400 rows x 6 columns]
